In [1]:
using ApproxFun

using Plots
gr() # GR high-efficiency graphing
#default(show=true)
#default(size=(1024,768)) # of rendered PNGs


Plots.GRBackend()

In [2]:
#pts=points(c,100) #This is a built in function; Approxfun would then do eg. vals=cos(pts) to get the values
#show(pts) # Density looks like a 'U' with greater sampling density near the extremes of the range

# From: https://github.com/ApproxFun/ApproxFun.jl/issues/275 , courtesy of private communication with Sheehan Olver
# Least squares approximation of data on an evenly spaced grid with Chebyshev series
function vandermonde(S,n,x::AbstractVector)
    V=Array{Float64}(length(x),n)
    for k=1:n
        #@printf("K-coeff: %d of %d\n",k,n)
        #println("Interval: ",[zeros(k-1);1.0] )
        #println(x)
        V[:,k]=Fun(S,[zeros(k-1);1.0]).(x)
    end
    V
end

function loadB(filename)
    c=Chebyshev(Interval(0,0.08)) #Define Chebyshev domain in this range (to match data imported)

    # Standard two column data form
    df=readdlm(filename)

    pts=df[:,1] # Points
    vals=df[:,2] #Values at these points

    println("pts as read in:")
    show(pts)

    # For ...(this)... case, make sure `length(pts) >> n`.
    n=13 # This is a magic number, found to give a good fit to Pooya's data
    println("Attempting Vandermonde / Chebyshev fit with: Range: ",c," Points in fit: ",n," Data points:",length(pts))
    V=vandermonde(c,n,pts)
    # Are you ready for the magic?
    af=Fun(c,V\vals) # Approximate Function (af)
    # me is now an ApproxFun representation of the tabulated data.
    # As a Chebyshev polynomial fit we can do all sorts of differentiation + integration.
    return af,df
end



loadB (generic function with 1 method)

In [21]:
#B,df=loadB("jagged.dat") # Only 8 points, very sawtoothy
B,df=loadB("/Users/jarvist/REPOS/B-coefficient-ApproxFun/B.cgs.kT-.0259.nh3ch3")
# Pooya's data runs from densities between 0 .. 0.08
# Unit is in electrons / unit cell, so multiply by ~4.03e21 to get cm^-3

# Print out the fit vs. the raw data, so you can see the residual
function graphB(af,df)
    # Logscale version...
    plot(af,label="Chebyshev (fit) B")
    plot!(df[:,1],df[:,2],label="Tabulated (input) B")
    yaxis!("B coeff")
    xaxis!("Density")
    png("Bcoeff_linear.png")

    xaxis!(:log10)
    png("Bcoeff_log.png")
end

graphB(B,df)

# OK; we have an ApproxFun function (B) fitted to the tabulated data

# B is our Approxfun fit; internally it's a polynomial, but you can differentiate etc. as if it were analytic



pts as read in:
[1.0e-9, 2.0e-9, 3.0e-9, 4.0e-9, 5.0e-9, 6.0e-9, 7.0e-9, 8.0e-9, 9.0e-9, 1.0e-8, 2.0e-8, 3.0e-8, 4.0e-8, 5.0e-8, 6.0e-8, 7.0e-8, 8.0e-8, 9.0e-8, 1.0e-7, 2.0e-7, 3.0e-7, 4.0e-7, 5.0e-7, 6.0e-7, 7.0e-7, 8.0e-7, 9.0e-7, 1.0e-6, 2.0e-6, 3.0e-6, 4.0e-6, 5.0e-6, 6.0e-6, 7.0e-6, 8.0e-6, 9.0e-6, 1.0e-5, 2.0e-5, 3.0e-5, 4.0e-5, 5.0e-5, 6.0e-5, 7.0e-5, 8.0e-5, 9.0e-5, 0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008, 0.0009, 0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.075, 0.076, 0.077, 0.078, 0.079]Attempting Vandermonde / Chebyshev fit with: Range: Chebyshev(【0.0,0.08】) Points in fit: 13 Data points:76


In [4]:
# We are now going to build our Ordinary Differential Equation model for n(t)
using ODE

type Model
    label
    ODE::Function # ODE of model
    t # holds calculated data
    xv
end

Models = [
    Model("Bcoeff",   (t,n) -> [ - B(n[1]/4.03e21) * n[1]*n[1] ; n[1]],[],[]), # Pure bimolecular; Pooya B(n)
#    Model("Bcoeff-A", (t,n) -> [-5e6*n[1] - B(n[1]/4.03e21) * n[1]*n[1] ; n[1]],[],[]), # SRH 'A' term from Herz; Pooya B(n)
    # DOI: 10.1021/acsenergylett.6b00236
#    Model("dQ-treatedfilm-A-B-C",   (t,n) -> [-3.8e4*n[1] - 4.0e-11*n[1]*n[1] - 4e-28*n[1]*n[1]*n[1] ; n[1]],[],[]),

    #Herz values from DOI: 10.1021/acs.accounts.5b00411
    #AH=5e6
    #BH=0.9e-10
#    Model("Herz-A-B", (t,n) -> [-5e6*n[1] - 0.9e-10 * n[1]*n[1]; n[1]],[],[]), # SRH 'A' term and bimolecular fit from above
#    Model("Herz-A",    (t,n) -> [-5e6*n[1]; n[1] ], [], []) # SRH 'A' term only
]

# Initial vector; density is first part
#initial=[0.08*4.03e21, 0.] # Start at n=0.08 Pooyas
initial=[1e19, 0.] # sensible laser fluence

for model in Models
    println("Simulating model: ",model.label)
    model.t,model.xv=ode23(model.ODE,initial,[0.0;900e-9]) # Numerically Integrate from 0 to ... seconds
    model.xv=hcat(model.xv...).' 
    #plotsoln(model.t,model.xv)
end

println("Plotting charge density vs. time")
plot()
yaxis!("Density n (1e18 cm-3)")
xaxis!("Time (ns)")
for model in Models
    plot!(model.t.*1e9,model.xv[:,1].*1e-18,label=model.label,ylims=(0.1,Inf))
end
png("density_linear.png")

yaxis!(:log10)
png("density_log.png")

println("Plotting integrated emission.")
plot()
for model in Models
    plot(model.t,model.xv[:,2],label=model.label) # Time on x-axis, versus n[2] (integrating dn/dt, emission) on Y axis
end
yaxis!("Integrated Emission (???)")
xaxis!("Time (s)")
png("integrated_emission.png")

println("Simulating radiative emission (TRPL) and plotting...")
plot()

I(n) = -B(n/4.03e21) * n*n # Emission model
for model in Models
    intensity=[-I(x) for x in model.xv[:,1]] #extract intensity as a function of time, by feeding the solved densities (from the ODE) into the solver
    # Nb: probably not the most elegant way to do this (!)
    intensity=hcat(intensity...).'

    # calculates intensity reusing the same functional toolkit - NB: assumes all recombination is emissive
    plot!(model.t,intensity[:,1],label=model.label)
end

yaxis!("Emission Intensity")
xaxis!("Time (s)")
png("emission.png")
yaxis!(:log10)
png("emission_log.png")

INFO: Recompiling stale cache file /Users/jarvist/.julia/lib/v0.6/SimpleTraits.ji for module SimpleTraits.
INFO: Recompiling stale cache file /Users/jarvist/.julia/lib/v0.6/ODE.ji for module ODE.


Simulating model: Bcoeff
Plotting charge density vs. time
Plotting integrated emission.
Simulating radiative emission (TRPL) and plotting...


In [18]:
# Load Pooya's IBSC B-coeff data
function loadRnp(filename)
    c=Chebyshev(Interval(0,0.08)) #Define Chebyshev domain in this range (to match data imported)

    # Standard two column data form
    df=readdlm(filename)

    n=df[:,1] # N (or possibly P)
    p=df[:,2] # P (or possibly N)
    R=df[:,3] # Rates

    # For ...(this)... case, make sure `length(pts) >> n`.
    N=13 # This is a magic number, found to give a good fit to Pooya's data
    println("Attempting Vandermonde / Chebyshev fit with: Range: ",c," Points in fit: ",N," Data points:",length(n))
    V=vandermonde(c,N,n)
    # Are you ready for the magic?
    af=Fun(c,V\R) # Approximate Function (af)
    # me is now an ApproxFun representation of the tabulated data.
    # As a Chebyshev polynomial fit we can do all sorts of differentiation + integration.
    return af,df
end

B,df=loadRnp("/Users/jarvist/REPOS/B-coefficient-ApproxFun/Rrate.kT-0.0259.IB-CB.mapi")

Attempting Vandermonde / Chebyshev fit with: Range: Chebyshev(【0.0,0.08】) Points in fit: 13 Data points:72


(Fun(Chebyshev(【0.0,0.08】),[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]), [8.06e13 8.06e11 … 2.0e-10 8.23308e-10; 8.06e13 8.06e12 … 2.0e-9 8.23087e-9; … ; 8.06e20 8.06e18 … 0.002 0.000171672; 8.06e20 8.06e19 … 0.02 0.00449533])

In [19]:
graphB(B,df)

In [35]:
# We are now going to build our Ordinary Differential Equation model for n(t)
using DifferentialEquations

println(B(0.08)) # starting value

# Intensity as function of time and density
Intensity(n,p,t) =  - B(n/4.03e21) * n*n 
#Intensity(n,p,t) =  - 8.394371185223479e-12 * n*n
#Intensity(u,p,t) = 0.98u
# Pure bimolecular; Pooya B(n) : - B(n/4.03e21) * n*n 
#  [-A*n[1] - B(n[1]/4.03e21) * n[1]*n[1] ; n[1]] # SRH 'A' term from above; Pooya B(n)
#  [-A*n[1] - Bconst * n[1]*n[1]; n[1] ] # SRH 'A' term and bimolecular fit from above

n0=0.08*4.03e21
tspan=(0,1e-9)

prob=ODEProblem(Intensity,n0,tspan)
sol=solve(prob)

plot(sol, title="Charge density", xaxis="Time (s)",yaxis="density (cm^-3)") # Plots recipe

8.394371185223479e-12


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0000000000 
 
 
 0.0000000002 
 
 
 0.0000000004 
 
 
 0.0000000006 
 
 
 0.0000000008 
 
 
 0.0000000010 
 
 
 100000000000000000000 
 
 
 150000000000000000000 
 
 
 200000000000000000000 
 
 
 250000000000000000000 
 
 
 300000000000000000000 
 
 
 Charge density 
 
 
 Time (s) 
 
 
 density (cm 
 
 
 - 
 
 
 3) 
 
<polyline clip-path="url(#clip4402)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 55.1181,27.5591 55.6557,28.8389 56.1932,30.1103 56.7307,31.3736 57.2683,32.6287 57.8058,33.8757 58.3434,35.1147 58.8809,36.3459 59.4185,37.5692 59.956,38.7848 
 60.4936,39.9928 61.0311,41.1932 61.5687,42.3862 62.1062,43.5718 62.6437,44.75 63.1813,45.9211 63.7188,47.085 64.2564,48.2418 64.7939,49.3916 65.3315,50.5345 
 65.869,51.6706 66.4066,52.8 66.9441,53.9226 67.4817,55.0387 68.0192,56.1482 68.5567,57.2513 69.0943,58.348 69.6318,59.4384 70.1694,60.5225 70.7069,61.6005 
 71.2445,62.6724 71.782,63.7383 72.3196,64.7982 72.8571,65.8522 73.3947,66.9004 73.9322,67.9428 74.4697,68.9795 75.0073,70.0107 75.5448,71.0363 76.0824,72.0563 
 76.6199,73.071 77.1575,74.0803 77.695,75.0842 78.2326,76.083 78.7701,77.0765 79.3077,78.065 79.8452,79.0484 80.3827,80.0267 80.9203,81.0001 81.4578,81.9687 
 81.9954,82.9324 82.5329,83.8913 83.0705,84.8455 83.608,85.7951 84.1456,86.74 84.6831,87.6804 85.2207,88.6162 85.7582,89.5476 86.2957,90.4747 86.8333,91.3973 
 87.3708,92.3157 87.9084,93.2298 88.4459,94.1397 88.9835,95.0454 89.521,95.947 90.0586,96.8446 90.5961,97.7381 91.1337,98.6277 91.6712,99.5133 92.2087,100.395 
 92.7463,101.273 93.2838,102.147 93.8214,103.017 94.3589,103.884 94.8965,104.747 95.434,105.606 95.9716,106.462 96.5091,107.314 97.0467,108.163 97.5842,109.008 
 98.1217,109.849 98.6593,110.688 99.1968,111.523 99.7344,112.354 100.272,113.182 100.809,114.007 101.347,114.829 101.885,115.647 102.422,116.462 102.96,117.274 
 103.497,118.083 104.035,118.889 104.572,119.692 105.11,120.492 105.647,121.288 106.185,122.082 106.722,122.873 107.26,123.66 107.798,124.445 108.335,125.227 
 108.873,126.006 109.41,126.782 109.948,127.556 110.485,128.326 111.023,129.094 111.56,129.859 112.098,130.621 112.635,131.38 113.173,132.137 113.711,132.891 
 114.248,133.642 114.786,134.391 115.323,135.137 115.861,135.88 116.398,136.62 116.936,137.358 117.473,138.094 118.011,138.826 118.548,139.556 119.086,140.284 
 119.624,141.009 120.161,141.731 120.699,142.451 121.236,143.168 121.774,143.883 122.311,144.595 122.849,145.304 123.386,146.011 123.924,146.716 124.461,147.417 
 124.999,148.117 125.537,148.814 126.074,149.508 126.612,150.199 127.149,150.889 127.687,151.575 128.224,152.259 128.762,152.941 129.299,153.62 129.837,154.296 
 130.374,154.97 130.912,155.641 131.45,156.309 131.987,156.975 132.525,157.639 133.062,158.3 133.6,158.958 134.137,159.613 134.675,160.266 135.212,160.917 
 135.75,161.564 136.287,162.209 136.825,162.851 137.363,163.491 137.9,164.127 138.438,164.761 138.975,165.393 139.513,166.021 140.05,166.647 140.588,167.27 
 141.125,167.89 141.663,168.507 142.2,169.121 142.738,169.733 143.276,170.341 143.813,170.947 144.351,171.55 144.888,172.149 145.426,172.746 145.963,173.34 
 146.501,173.931 147.038,174.52 147.576,175.107 148.113,175.691 148.651,176.273 149.189,176.852 149.726,177.429 150.264,178.004 150.801,178.576 151.339,179.146 
 151.876,179.714 152.414,180.279 152.951,180.843 153.489,181.404 154.026,181.962 154.564,182.519 155.102,183.073 155.639,183.625 156.177,184.175 156.714,184.723 
 157.252,185.269 157.789,185.812 158.327,186.354 158.864,186.893 159.402,187.431 159.939,187.966 160.477,188.499 161.015,189.03 161.552,189.56 162.09,190.087 
 162.627,190.612 163.165,191.135 163.702,191.657 164.24,192.176 164.777,192.693 165.315,193.209 165.852,193.723 166.39,194.234 166.928,194.744 167.465,195.252 
 168.003,195.758 168.54,196.263 169.078,196.765 169.615,197.266 170.153,197.765 170.69,1

In [119]:
function IBSC(dn,n,p,t)
    const A=4e+4 #  Light fluence * matrix element^2 ?
    # Valence band
    dn[1]= -A * n[1]
    # Intermediate band
    dn[2]= A*n[1] - B(n[2]/4.03e21) * n[2]*n[2] - A*n[2]
    # Conduction band
    dn[3]= A*n[2] # crosssection to conduction band
end


IBSC (generic function with 1 method)

In [120]:
n0 = [1.0;0.0;0.0] # start with all density in valence band
tspan=(0,1e-4)
prob = ODEProblem(IBSC,n0,tspan)
sol=solve(prob)

retcode: Success
Interpolation: specialized 4th order "free" interpolation
t: 16-element Array{Float64,1}:
 0.0       
 2.4975e-8 
 2.74725e-7
 1.31211e-6
 3.08843e-6
 5.58312e-6
 9.00919e-6
 1.35118e-5
 1.92664e-5
 2.63887e-5
 3.49573e-5
 4.50366e-5
 5.66481e-5
 6.97623e-5
 8.43152e-5
 0.0001    
u: 16-element Array{Array{Float64,1},1}:
 [1.0, 0.0, 0.0]                    
 [0.999001, 0.000998003, 4.98669e-7]
 [0.989071, 0.0108689, 5.99386e-5]  
 [0.948869, 0.0498007, 0.00133004]  
 [0.883789, 0.109181, 0.00703045]   
 [0.799855, 0.178628, 0.0215175]    
 [0.69742, 0.251328, 0.0512526]     
 [0.582473, 0.314811, 0.102716]     
 [0.462709, 0.35659, 0.180701]      
 [0.348002, 0.367332, 0.284665]     
 [0.247018, 0.345403, 0.407578]     
 [0.165057, 0.297344, 0.537599]     
 [0.103735, 0.235052, 0.661213]     
 [0.0613915, 0.171309, 0.7673]      
 [0.0343007, 0.115678, 0.850021]    
 [0.0183163, 0.0732604, 0.908423]   

In [121]:
plot(sol,vars=(0,1),label="Valence Band")
plot!(sol,vars=(0,2),label="Intermediate Band")
plot!(sol,vars=(0,3),label="Conduction Band")


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00000 
 
 
 0.00002 
 
 
 0.00004 
 
 
 0.00006 
 
 
 0.00008 
 
 
 0.00010 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 t 
 
<polyline clip-path="url(#clip5702)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,7.87402 39.9234,9.33082 40.4767,10.7818 41.03,12.227 41.5833,13.6664 42.1366,15.1001 42.6899,16.528 43.2432,17.9502 43.7966,19.3668 44.3499,20.7777 
 44.9032,22.1829 45.4565,23.5825 46.0098,24.9766 46.5631,26.365 47.1164,27.748 47.6697,29.1254 48.223,30.4972 48.7763,31.8637 49.3296,33.2246 49.883,34.5801 
 50.4363,35.9302 50.9896,37.2749 51.5429,38.6142 52.0962,39.9482 52.6495,41.2768 53.2028,42.6002 53.7561,43.9182 54.3094,45.231 54.8627,46.5385 55.416,47.8408 
 55.9694,49.1379 56.5227,50.4298 57.076,51.7166 57.6293,52.9982 58.1826,54.2747 58.7359,55.5461 59.2892,56.8124 59.8425,58.0737 60.3958,59.3299 60.9491,60.5811 
 61.5024,61.8273 62.0558,63.0685 62.6091,64.3047 63.1624,65.536 63.7157,66.7624 64.269,67.9839 64.8223,69.2005 65.3756,70.4123 65.9289,71.6192 66.4822,72.8213 
 67.0355,74.0185 67.5888,75.211 68.1422,76.3988 68.6955,77.5817 69.2488,78.76 69.8021,79.9336 70.3554,81.1024 70.9087,82.2666 71.462,83.4261 72.0153,84.581 
 72.5686,85.7313 73.1219,86.877 73.6753,88.0181 74.2286,89.1547 74.7819,90.2867 75.3352,91.4142 75.8885,92.5372 76.4418,93.6557 76.9951,94.7697 77.5484,95.8793 
 78.1017,96.9844 78.655,98.0851 79.2083,99.1815 79.7617,100.273 80.315,101.361 80.8683,102.444 81.4216,103.523 81.9749,104.598 82.5282,105.668 83.0815,106.734 
 83.6348,107.796 84.1881,108.853 84.7414,109.907 85.2947,110.956 85.8481,112.001 86.4014,113.041 86.9547,114.078 87.508,115.11 88.0613,116.139 88.6146,117.163 
 89.1679,118.183 89.7212,119.199 90.2745,120.211 90.8278,121.219 91.3811,122.223 91.9345,123.223 92.4878,124.218 93.0411,125.21 93.5944,126.198 94.1477,127.182 
 94.701,128.162 95.2543,129.138 95.8076,130.111 96.3609,131.079 96.9142,132.043 97.4675,133.004 98.0209,133.961 98.5742,134.914 99.1275,135.863 99.6808,136.808 
 100.234,137.75 100.787,138.688 101.341,139.622 101.894,140.552 102.447,141.479 103.001,142.402 103.554,143.321 104.107,144.237 104.661,145.148 105.214,146.057 
 105.767,146.961 106.32,147.862 106.874,148.76 107.427,149.654 107.98,150.544 108.534,151.431 109.087,152.314 109.64,153.193 110.194,154.069 110.747,154.942 
 111.3,155.811 111.854,156.677 112.407,157.539 112.96,158.398 113.514,159.253 114.067,160.105 114.62,160.954 115.173,161.799 115.727,162.64 116.28,163.479 
 116.833,164.314 117.387,165.145 117.94,165.974 118.493,166.799 119.047,167.621 119.6,168.439 120.153,169.254 120.707,170.066 121.26,170.875 121.813,171.68 
 122.366,172.483 122.92,173.282 123.473,174.077 124.026,174.87 124.58,175.66 125.133,176.446 125.686,177.229 126.24,178.009 126.793,178.786 127.346,179.56 
 127.9,180.331 128.453,181.098 129.006,181.863 129.559,182.625 130.113,183.383 130.666,184.139 131.219,184.891 131.773,185.64 132.326,186.387 132.879,187.13 
 133.433,187.871 133.986,188.608 134.539,189.343 135.093,190.075 135.646,190.803 136.199,191.529 136.753,192.252 137.306,192.972 137.859,193.689 138.412,194.404 
 138.966,195.115 139.519,195.824 140.072,196.529 140.626,197.232 141.179,197.932 141.732,198.63 142.286,199.324 142.839,200.016 143.392,200.705 143.946,201.391 
 144.499,202.075 145.052,202.756 145.605,203.434 146.159,204.109 146.712,204.782 147.265,205.452 147.819,206.119 148.372,206.784 148.925,207.446 149.479,208.105 
 150.032,208.762 150.585,209.416 151.139,210.067 151.692,210.716 152.245,211.362 152.798,212.006 153.352,212.647 153.905,213.286 154.458,213.922 155.012,214.555 
 155.565,215.186 156.118,215.814 156.672,216.44 157.225,217.064 157.778,217.684 158.332,218.303 158.885,218.919 159.438,219.532 159.991,220.143 160.545,220.752 
 161.098,221.358 161.651,221.962 162.205,222.563 162.758,223.162 163.311,223.758 163.865,224.353

In [122]:
# IT WORKS! :^)

In [196]:
# Broken attempt at trying to put in all the little matrix elements *n*p for elevation

function IBSC(dn,n,p,t)
    const A=4e4 # Light fluence * matrix element^2 ?. Should be counts/s units.
    # Valence band
    dn[1]= -A*n[1]*(1-n[2]) + A*n[2]*(1-n[1]) + B(n[2]/4.03e21)*n[2]*(1-n[1]) 
    # Intermediate band
    dn[2]= A*n[1]*(1-n[2]) + A*n[3]*(1-n[2]) - 
         A*n[2]*(1-n[1]) - A*n[2]*(1-n[3]) -
         B(n[2]/4.03e21)*n[2]*(1-n[1]) #- A*n[2]
    # Conduction band
    dn[3]= A*n[2]*(1-n[3]) - A*n[3]*(1-n[2]) # crosssection to conduction band
end

n0 = [1.0;0.0;0.0] # start with all density in valence band
tspan=(0,1e-3)
prob = ODEProblem(IBSC,n0,tspan)
sol=solve(prob)

plot()
plot!(sol,vars=(0,1),label="Valence Band")

plot!(sol,vars=(0,2),label="Intermediate Band")
plot!(sol,vars=(0,3),label="Conduction Band")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0000 
 
 
 0.0002 
 
 
 0.0004 
 
 
 0.0006 
 
 
 0.0008 
 
 
 0.0010 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 t 
 
<polyline clip-path="url(#clip5902)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,7.87402 39.9234,21.9058 40.4767,34.8783 41.03,46.8908 41.5833,58.0319 42.1366,68.381 42.6899,78.0095 43.2432,86.9812 43.7966,95.3534 44.3499,103.178 
 44.9032,110.5 45.4565,117.363 46.0098,123.804 46.5631,129.856 47.1164,135.55 47.6697,140.914 48.223,145.973 48.7763,150.749 49.3296,155.264 49.883,159.535 
 50.4363,163.58 50.9896,167.415 51.5429,171.053 52.0962,174.507 52.6495,177.79 53.2028,180.911 53.7561,183.882 54.3094,186.711 54.8627,189.407 55.416,191.977 
 55.9694,194.428 56.5227,196.767 57.076,199.001 57.6293,201.135 58.1826,203.174 58.7359,205.124 59.2892,206.989 59.8425,208.772 60.3958,210.479 60.9491,212.113 
 61.5024,213.677 62.0558,215.176 62.6091,216.611 63.1624,217.987 63.7157,219.305 64.269,220.569 64.8223,221.781 65.3756,222.942 65.9289,224.056 66.4822,225.124 
 67.0355,226.149 67.5888,227.132 68.1422,228.075 68.6955,228.981 69.2488,229.85 69.8021,230.684 70.3554,231.484 70.9087,232.252 71.462,232.99 72.0153,233.698 
 72.5686,234.377 73.1219,235.029 73.6753,235.656 74.2286,236.257 74.7819,236.835 75.3352,237.39 75.8885,237.923 76.4418,238.435 76.9951,238.927 77.5484,239.399 
 78.1017,239.852 78.655,240.287 79.2083,240.706 79.7617,241.107 80.315,241.493 80.8683,241.863 81.4216,242.219 81.9749,242.561 82.5282,242.889 83.0815,243.204 
 83.6348,243.508 84.1881,243.799 84.7414,244.079 85.2947,244.348 85.8481,244.606 86.4014,244.854 86.9547,245.093 87.508,245.322 88.0613,245.541 88.6146,245.752 
 89.1679,245.955 89.7212,246.15 90.2745,246.337 90.8278,246.517 91.3811,246.689 91.9345,246.855 92.4878,247.015 93.0411,247.168 93.5944,247.315 94.1477,247.457 
 94.701,247.593 95.2543,247.724 95.8076,247.849 96.3609,247.97 96.9142,248.086 97.4675,248.197 98.0209,248.304 98.5742,248.407 99.1275,248.505 99.6808,248.6 
 100.234,248.691 100.787,248.778 101.341,248.862 101.894,248.943 102.447,249.02 103.001,249.094 103.554,249.166 104.107,249.235 104.661,249.301 105.214,249.364 
 105.767,249.425 106.32,249.484 106.874,249.54 107.427,249.595 107.98,249.647 108.534,249.697 109.087,249.745 109.64,249.791 110.194,249.836 110.747,249.878 
 111.3,249.919 111.854,249.958 112.407,249.996 112.96,250.032 113.514,250.067 114.067,250.1 114.62,250.132 115.173,250.163 115.727,250.193 116.28,250.221 
 116.833,250.248 117.387,250.274 117.94,250.299 118.493,250.324 119.047,250.347 119.6,250.369 120.153,250.391 120.707,250.412 121.26,250.432 121.813,250.451 
 122.366,250.469 122.92,250.487 123.473,250.504 124.026,250.521 124.58,250.536 125.133,250.552 125.686,250.566 126.24,250.58 126.793,250.594 127.346,250.606 
 127.9,250.619 128.453,250.63 129.006,250.642 129.559,250.653 130.113,250.663 130.666,250.673 131.219,250.682 131.773,250.691 132.326,250.7 132.879,250.709 
 133.433,250.717 133.986,250.724 134.539,250.732 135.093,250.739 135.646,250.746 136.199,250.753 136.753,250.759 137.306,250.765 137.859,250.771 138.412,250.777 
 138.966,250.783 139.519,250.788 140.072,250.794 140.626,250.799 141.179,250.804 141.732,250.809 142.286,250.813 142.839,250.817 143.392,250.822 143.946,250.826 
 144.499,250.83 145.052,250.833 145.605,250.837 146.159,250.84 146.712,250.843 147.265,250.846 147.819,250.849 148.372,250.852 148.925,250.854 149.479,250.857 
 150.032,250.859 150.585,250.861 151.139,250.863 151.692,250.865 152.245,250.867 152.798,250.869 153.352,250.87 153.905,250.872 154.458,250.874 155.012,250.875 
 155.565,250.877 156.118,250.878 156.672,250.879 157.225,250.881 157.778,250.882 158.332,250.884 158.885,250.885 159.438,250.887 159.991,250.889 160.545,250.892 
 161.098,250.894 161.651,250.897 162.205,250.899 162.758,250.902 163.311,250.904 163.865,250.906 164.4

In [198]:
# Attempt at simplifying.

myB(n)=1e15 * 8.394371185223479e-12

function IBSC(dn,n,p,t)
    const A=1e3 # Light fluence * matrix element^2 ?. Should be counts/s units.
    # Valence band
    dn[1]= -A*n[1] + myB(n[2])*n[2]*(1-n[1]) 
    # Intermediate band
    dn[2]= A*n[1] - myB(n[2])*n[2]*(1-n[1]) #- A*n[2]
    # Conduction band
    dn[3]= 0 #A*n[2]*(1-n[3]) - A*n[3]*(1-n[2]) # crosssection to conduction band
end

n0 = [1.0;0.0;0.0] # start with all density in valence band
tspan=(0,1e-2)
prob = ODEProblem(IBSC,n0,tspan)
sol=solve(prob)

plot()
plot!(sol,vars=(0,1),label="Valence Band")
plot!(sol,vars=(0,2),label="Intermediate Band")
plot!(sol,vars=(0,3),label="Conduction Band")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.000 
 
 
 0.002 
 
 
 0.004 
 
 
 0.006 
 
 
 0.008 
 
 
 0.010 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 t 
 
<polyline clip-path="url(#clip6302)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,7.87402 39.9234,11.5041 40.4767,15.0921 41.03,18.6325 41.5833,22.1203 42.1366,25.5508 42.6899,28.9196 43.2432,32.2227 43.7966,35.4564 44.3499,38.6174 
 44.9032,41.7029 45.4565,44.7105 46.0098,47.6379 46.5631,50.4836 47.1164,53.2462 47.6697,55.9247 48.223,58.5185 48.7763,61.0271 49.3296,63.4506 49.883,65.7893 
 50.4363,68.0438 50.9896,70.2147 51.5429,72.3033 52.0962,74.3106 52.6495,76.2381 53.2028,78.0873 53.7561,79.8597 54.3094,81.557 54.8627,83.1811 55.416,84.7341 
 55.9694,86.218 56.5227,87.6351 57.076,88.9876 57.6293,90.2776 58.1826,91.5073 58.7359,92.6787 59.2892,93.7939 59.8425,94.8551 60.3958,95.8644 60.9491,96.8239 
 61.5024,97.7355 62.0558,98.6014 62.6091,99.4236 63.1624,100.204 63.7157,100.945 64.269,101.648 64.8223,102.314 65.3756,102.946 65.9289,103.544 66.4822,104.111 
 67.0355,104.648 67.5888,105.156 68.1422,105.636 68.6955,106.091 69.2488,106.521 69.8021,106.928 70.3554,107.314 70.9087,107.678 71.462,108.023 72.0153,108.349 
 72.5686,108.658 73.1219,108.949 73.6753,109.224 74.2286,109.484 74.7819,109.73 75.3352,109.962 75.8885,110.181 76.4418,110.387 76.9951,110.582 77.5484,110.766 
 78.1017,110.94 78.655,111.104 79.2083,111.26 79.7617,111.406 80.315,111.545 80.8683,111.676 81.4216,111.8 81.9749,111.916 82.5282,112.026 83.0815,112.13 
 83.6348,112.228 84.1881,112.32 84.7414,112.407 85.2947,112.489 85.8481,112.566 86.4014,112.638 86.9547,112.707 87.508,112.771 88.0613,112.832 88.6146,112.89 
 89.1679,112.944 89.7212,112.996 90.2745,113.045 90.8278,113.091 91.3811,113.134 91.9345,113.175 92.4878,113.214 93.0411,113.251 93.5944,113.285 94.1477,113.317 
 94.701,113.348 95.2543,113.376 95.8076,113.403 96.3609,113.428 96.9142,113.452 97.4675,113.475 98.0209,113.496 98.5742,113.515 99.1275,113.534 99.6808,113.552 
 100.234,113.569 100.787,113.585 101.341,113.6 101.894,113.614 102.447,113.628 103.001,113.641 103.554,113.653 104.107,113.664 104.661,113.675 105.214,113.685 
 105.767,113.695 106.32,113.704 106.874,113.712 107.427,113.72 107.98,113.727 108.534,113.734 109.087,113.741 109.64,113.747 110.194,113.752 110.747,113.758 
 111.3,113.763 111.854,113.767 112.407,113.772 112.96,113.776 113.514,113.78 114.067,113.783 114.62,113.787 115.173,113.791 115.727,113.794 116.28,113.797 
 116.833,113.8 117.387,113.803 117.94,113.806 118.493,113.809 119.047,113.811 119.6,113.813 120.153,113.816 120.707,113.818 121.26,113.819 121.813,113.821 
 122.366,113.823 122.92,113.824 123.473,113.826 124.026,113.827 124.58,113.828 125.133,113.829 125.686,113.83 126.24,113.831 126.793,113.832 127.346,113.833 
 127.9,113.834 128.453,113.834 129.006,113.835 129.559,113.836 130.113,113.836 130.666,113.837 131.219,113.837 131.773,113.838 132.326,113.839 132.879,113.839 
 133.433,113.84 133.986,113.84 134.539,113.841 135.093,113.841 135.646,113.842 136.199,113.842 136.753,113.843 137.306,113.843 137.859,113.844 138.412,113.844 
 138.966,113.845 139.519,113.845 140.072,113.845 140.626,113.845 141.179,113.846 141.732,113.846 142.286,113.846 142.839,113.846 143.392,113.846 143.946,113.847 
 144.499,113.847 145.052,113.847 145.605,113.847 146.159,113.847 146.712,113.847 147.265,113.847 147.819,113.847 148.372,113.847 148.925,113.847 149.479,113.847 
 150.032,113.847 150.585,113.847 151.139,113.847 151.692,113.847 152.245,113.847 152.798,113.847 153.352,113.847 153.905,113.847 154.458,113.847 155.012,113.847 
 155.565,113.847 156.118,113.848 156.672,113.848 157.225,113.848 157.778,113.848 158.332,113.848 158.885,113.848 159.438,113.848 159.991,113.848 160.545,113.848 
 161.098,113.848 161.651,113.849 162.205,113.849 162.758,113.849 163.311,113.849 163.865,113.849 164.41

In [213]:
# Attempt at simplifying.

myB(n)=1e15 * 8.394371185223479e-12

function IBSC(dn,n,p,t)
    const L=1e2 # Light fluence * matrix element^2 ?. Should be counts/s units.
    const Spon=0 #1e5 # Spontaneous down-emission
    # Valence band
    dn[1]= -L*n[1] + myB(n[2])*n[2]*(1-n[1]) + Spon*n[3]
    # Intermediate band
    dn[2]= L*n[1] - myB(n[2])*n[2]*(1-n[1]) - L*n[2] 
    # Conduction band
    dn[3]= L*n[2] - Spon*n[3]# crosssection to conduction band
end

n0 = [1.0;0.0;0.0] # start with all density in valence band
tspan=(0,1e-1)
prob = ODEProblem(IBSC,n0,tspan)
sol=solve(prob)

plot()
plot!(sol,vars=(0,1),label="Valence Band")
plot!(sol,vars=(0,2),label="Intermediate Band")
plot!(sol,vars=(0,3),label="Conduction Band")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 0.08 
 
 
 0.10 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 t 
 
<polyline clip-path="url(#clip9302)" style="stroke:#009af9; stroke-width:3; stroke-opacity:1; fill:none" points="
 39.3701,7.87402 39.9234,11.4951 40.4767,15.0215 41.03,18.4018 41.5833,21.594 42.1366,24.5672 42.6899,27.3021 43.2432,29.7898 43.7966,32.0306 44.3499,34.0328 
 44.9032,35.8095 45.4565,37.377 46.0098,38.7551 46.5631,39.9636 47.1164,41.0216 47.6697,41.9477 48.223,42.7597 48.7763,43.4731 49.3296,44.1014 49.883,44.657 
 50.4363,45.1513 50.9896,45.5943 51.5429,45.9933 52.0962,46.3549 52.6495,46.6851 53.2028,46.9893 53.7561,47.2722 54.3094,47.5368 54.8627,47.7857 55.416,48.0214 
 55.9694,48.2462 56.5227,48.4622 57.076,48.6712 57.6293,48.8739 58.1826,49.0713 58.7359,49.264 59.2892,49.4529 59.8425,49.6386 60.3958,49.8219 60.9491,50.0033 
 61.5024,50.1828 62.0558,50.3608 62.6091,50.5374 63.1624,50.7127 63.7157,50.8869 64.269,51.0603 64.8223,51.2331 65.3756,51.4053 65.9289,51.577 66.4822,51.7483 
 67.0355,51.9191 67.5888,52.0895 68.1422,52.2596 68.6955,52.4294 69.2488,52.5988 69.8021,52.768 70.3554,52.937 70.9087,53.1057 71.462,53.2738 72.0153,53.4405 
 72.5686,53.606 73.1219,53.7709 73.6753,53.9357 74.2286,54.1007 74.7819,54.2662 75.3352,54.4324 75.8885,54.5993 76.4418,54.7671 76.9951,54.9357 77.5484,55.105 
 78.1017,55.2748 78.655,55.4449 79.2083,55.6148 79.7617,55.7843 80.315,55.9528 80.8683,56.1197 81.4216,56.2844 81.9749,56.4468 82.5282,56.6088 83.0815,56.771 
 83.6348,56.9332 84.1881,57.0957 84.7414,57.2584 85.2947,57.4213 85.8481,57.5843 86.4014,57.7474 86.9547,57.9104 87.508,58.0731 88.0613,58.2353 88.6146,58.3968 
 89.1679,58.5575 89.7212,58.7179 90.2745,58.8781 90.8278,59.0384 91.3811,59.1988 91.9345,59.3594 92.4878,59.5204 93.0411,59.6816 93.5944,59.843 94.1477,60.0045 
 94.701,60.1658 95.2543,60.3269 95.8076,60.4874 96.3609,60.6469 96.9142,60.8053 97.4675,60.9628 98.0209,61.1198 98.5742,61.2767 99.1275,61.4335 99.6808,61.5904 
 100.234,61.7476 100.787,61.9051 101.341,62.0628 101.894,62.2208 102.447,62.3788 103.001,62.5369 103.554,62.6946 104.107,62.8518 104.661,63.0082 105.214,63.1633 
 105.767,63.3175 106.32,63.471 106.874,63.6242 107.427,63.7773 107.98,63.9306 108.534,64.0842 109.087,64.2382 109.64,64.3926 110.194,64.5473 110.747,64.7023 
 111.3,64.8575 111.854,65.0125 112.407,65.1672 112.96,65.3212 113.514,65.4741 114.067,65.6255 114.62,65.7756 115.173,65.9251 115.727,66.0743 116.28,66.2236 
 116.833,66.373 117.387,66.5229 117.94,66.6732 118.493,66.8241 119.047,66.9755 119.6,67.1273 120.153,67.2793 120.707,67.4312 121.26,67.5828 121.813,67.7336 
 122.366,67.8832 122.92,68.031 123.473,68.1775 124.026,68.3231 124.58,68.4683 125.133,68.6136 125.686,68.7592 126.24,68.9053 126.793,69.052 127.346,69.1994 
 127.9,69.3474 128.453,69.4959 129.006,69.6448 129.559,69.7936 130.113,69.9421 130.666,70.0899 131.219,70.2363 131.773,70.3809 132.326,70.5236 132.879,70.6655 
 133.433,70.8069 133.986,70.9484 134.539,71.0902 135.093,71.2325 135.646,71.3757 136.199,71.5196 136.753,71.6642 137.306,71.8095 137.859,71.9552 138.412,72.101 
 138.966,72.2465 139.519,72.3911 140.072,72.5344 140.626,72.6756 141.179,72.8148 141.732,72.9531 142.286,73.0908 142.839,73.2286 143.392,73.3668 143.946,73.5057 
 144.499,73.6454 145.052,73.786 145.605,73.9275 146.159,74.0696 146.712,74.2122 147.265,74.355 147.819,74.4973 148.372,74.6388 148.925,74.7788 149.479,74.9164 
 150.032,75.0521 150.585,75.1867 151.139,75.3211 151.692,75.4555 152.245,75.5904 152.798,75.7262 153.352,75.8628 153.905,76.0004 154.458,76.1389 155.012,76.2781 
 155.565,76.4177 156.118,76.5574 156.672,76.6966 157.225,76.8347 157.778,76.9709 158.332,77.1046 158.885,77.2366 159.438,77.3679 159.991,77.499 160.545,77.6305 
 161.098,77.7626 161.651,77.8956 162.205,78.0295 162.758,78.1645 163.311,78.3003 163.865,78.4368 164.418,78.5

8.394371185223479e-12